<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/SIMULACI%C3%93_A/Din%C3%A0mica_planet%C3%A0ria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

L'equació diferencial que hem de resoldre és de la forma:

$\frac{d^2\vec{r}_{i}}{dt^2}m_{i}=-Gm_{i}\sum{\frac{\vec{r}_{j}-\vec{r}_{i}}{|\vec{r}_{j}-\vec{r}_{i}|^3}m_{j}}$

On "i" és l'índex del cos que rep la força i "j" la dels cossos que fan força sobre ell.

Normalitzant per la Terra, queda:
$
\frac{d^2\vec{\tilde{r}}_{terra}}{d\tilde{t}^2}=-\left[\frac{\vec{\tilde{r}}_{sol}-\vec{\tilde{r}}_{terra}}{|\vec{\tilde{r}}_{sol}-\vec{\tilde{r}}_{terra}|^3}+
\frac{\tilde{m}_{lluna}}{\tilde{m}_{sol}}\frac{\vec{\tilde{r}}_{lluna}-\vec{\tilde{r}}_{terra}}{|\vec{\tilde{r}}_{lluna}-\vec{\tilde{r}}_{terra}|^3}+
\frac{\vec{\tilde{r}}_{mart}-\vec{\tilde{r}}_{terra}}{|\vec{\tilde{r}}_{mart}-\vec{\tilde{r}}_{terra}|^3}\right]
$